# Online Sales Data Analysis

## Importing Necessary Libraries

In [ ]:
# Importing all the required libraries for data manipulation, visualization, and analysis.
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from mlxtend.frequent_patterns import apriori, association_rules
from scipy.stats import mstats

In [ ]:
# Connect to Drive
from google.colab import drive
drive.mount('/content/drive')

## Data Preprocessing

### One Hot Encoding for Countries
#### We start by reading in the CSV file and performing one hot encoding on the 'Country' column to prepare the data for further analysis.

In [ ]:
# Load data and encode the 'Country' column into numerical values.
file_path = '/content/drive/MyDrive/OnlineRetail.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Display the unique countries in the 'Country' column
unique_countries = df['Country'].unique()

# Generate the country encoding dictionary
country_encoding = {country: index + 1 for index, country in enumerate(unique_countries)}

# Encode the 'Country' column using the generated country encoding dictionary
df['Country_Encoded'] = df['Country'].map(country_encoding)

# Rename UnitPrice to Price
df.rename(columns={'UnitPrice': 'Price'}, inplace=True)

# Convert 'Quantity' and 'Price' columns to numeric, setting errors='coerce' to handle non-numeric values ( 0nly for II)
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# Calculate the total value for each item by multiplying 'Quantity' with 'Price'
df['Total'] = df['Quantity'] * df['Price']


# Save the modified DataFrame to a new CSV file
output_file_path = '/content/online_retail_updated.csv'
df.to_csv(output_file_path, index=False)

## Data overview

### Checking and Preparing the Data
#### We load the updated CSV and perform basic data overview operations like displaying the first few rows and getting summary information about the data.


In [ ]:
# Load the updated CSV file with one-hot encoding.
retail = pd.read_csv('/content/online_retail_updated.csv')

# Display the first few rows of the dataset to verify the content.
retail.head()

In [ ]:
# Display basic information about the dataset to understand its structure.
retail.info()

In [ ]:
# Display all countries and the encoding
country_encoding

## Apriori Algorithm for Market Basket Analysis
#### We apply the Apriori algorithm to find frequent itemsets and generate association rules.

In [ ]:
# Grouping the data to convert into transaction format for Apriori
# Converting data into transaction format suitable for applying Apriori algorithm.
transactions = retail.groupby(['Quantity', 'Country_Encoded'])['Quantity'].count().unstack().reset_index().fillna(0).set_index('Quantity')

# Converting the data into boolean format for analysis.
transactions_boolean = transactions > 0

# Applying Apriori Algorithm to find frequent itemsets.
frequent_itemsets = apriori(transactions_boolean, min_support=0.05, use_colnames=True)

# Generating association rules based on the frequent itemsets found.
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules

## Interquartile Range (IQR) Calculation for Outlier Detection
#### We calculate the IQR for each numerical column to understand the spread of the data and detect potential outliers.


In [ ]:
# Select all numerical columns for analysis.
numerical_columns = retail.select_dtypes(include=['int64', 'float64']).columns
# Calculate Q1 (25th percentile) and Q3 (75th percentile) for each numerical column.
Q1 = retail[numerical_columns].quantile(0.25)
Q3 = retail[numerical_columns].quantile(0.75)
# Calculate IQR to detect potential outliers.
IQR = Q3 - Q1
print("Interquartile Range (IQR) for each numerical column:")
IQR

## Winsorization for Outlier Handling
#### Winsorization is performed to handle outliers by capping extreme values at the specified percentiles.


In [ ]:
# Create a temp variable to strore retail
temp = retail.copy()

# Define limits for Winsorization to cap extreme values in the dataset.
lower_limit = 0.05
upper_limit = 0.95
# Apply Winsorization to handle extreme outliers.
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
temp[numerical_columns] = temp[numerical_columns].apply(lambda x: mstats.winsorize(x, limits=(lower_limit, upper_limit)))
temp.head()

## General Sales Analytics
### Most and Least Expensive Product
#### We identify the most and least expensive products based on price.

In [ ]:
# Identify the product with the highest price.
most_expensive = retail.loc[retail['Price'] == retail['Price'].max()]
most_expensive

In [ ]:
# Identify the product with the lowest price and aggregate quantities for better understanding.
least_expensive = retail.loc[retail['Price'] == retail['Price'].min()]
least_expensive


## Customer Analysis
### Total Purchase Amount
#### We analyze customers based on the total amount they have spent to determine the most and least valuable clients.


In [ ]:
# Group by customers to calculate the total amount spent by each customer.
retail_customers = retail.groupby(['Customer ID', 'Country'], as_index=False)['Total'].agg('sum')
# Get the customers with the least total purchases.
retail_customers_asc = retail_customers.sort_values('Total').head(10)
# Get the customers with the highest total purchases.
retail_customers_desc = retail_customers.sort_values('Total', ascending=False).head(10)

#### Plotting Customers with Max/Min Total Purchase Amount


In [ ]:
# Plotting the customers with the maximum total purchase amount.
fig = go.Figure(data=[go.Bar(name='Customers with Max Total Purchase Amount', x=retail_customers_desc['Customer ID'].astype(str), y=retail_customers_desc['Total'], marker_opacity=1, hovertext=retail_customers_desc['Country'], marker={'color': retail_customers_desc['Total'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Customers with Max Total Purchase Amount', title_x=0.45, xaxis_title="Customer ID", yaxis_title="Total Amount, £", plot_bgcolor='white')
fig.show()

# Plotting the customers with the minimum total purchase amount.
fig = go.Figure(data=[go.Bar(name='Customers with Min Total Purchase Amount', x=retail_customers_asc['Customer ID'].astype(str), y=retail_customers_asc['Total'], marker_opacity=1, hovertext=retail_customers_asc['Country'], marker={'color': retail_customers_asc['Total'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Customers with Min Total Purchase Amount', title_x=0.45, xaxis_title="Customer ID", yaxis_title="Total Amount Returned, £", plot_bgcolor='white')
fig.show()

# Country Analysis

### Total Purchase Amount by Country
#### We find the top 10 countries with the highest and lowest total purchase amounts.


In [ ]:
## Group by countries to calculate the total amount of purchases for each country.
retail_countries_purchase = retail.groupby(['Country'], as_index=False)['Total'].agg('sum')
# Get the countries with the least total purchases.
retail_countries_purchase_asc = retail_countries_purchase.sort_values('Total').head(10)
# Get the countries with the highest total purchases.
retail_countries_purchase_desc = retail_countries_purchase.sort_values('Total', ascending=False).head(10)

#### Plotting Countries with Max/Min Total Purchase Amount


In [ ]:
# Plotting the countries with the maximum total purchase amount.
fig = go.Figure(data=[go.Bar(name='Countries with Max Total Purchase Amount', x=retail_countries_purchase_desc['Country'].astype(str), y=retail_countries_purchase_desc['Total'], marker_opacity=1, marker={'color': retail_countries_purchase_desc['Total'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Countries with Max Total Purchase Sum', title_x=0.45, xaxis_title="Countries", yaxis_title="Total Amount, £", plot_bgcolor='white')
fig.show()

# Plotting the countries with the minimum total purchase amount.
fig = go.Figure(data=[go.Bar(name='Countries with Min Total Purchase Amount', x=retail_countries_purchase_asc['Country'].astype(str), y=retail_countries_purchase_asc['Total'], marker_opacity=1, marker={'color': retail_countries_purchase_asc['Total'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Countries with Min Total Purchase Amount', title_x=0.45, xaxis_title="Countries", yaxis_title="Total Amount, £", plot_bgcolor='white')
fig.show()

### Unique Customers by Country
#### We analyze which countries have the largest and smallest numbers of unique customers.


In [ ]:
# Group by countries to find the number of unique customers from each country.
retail_countries_customers = retail.groupby(['Country'], as_index=False)['Customer ID'].agg({'Customer ID': 'nunique'})
retail_countries_customers.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)

# Finding countries with the largest and smallest number of unique customers.
retail_countries_customers_asc = retail_countries_customers.sort_values('Customer Count').head(10)
retail_countries_customers_desc = retail_countries_customers.sort_values('Customer Count', ascending=False).head(10)

#### Plotting Countries with Largest/Smallest Number of Unique Customers


In [ ]:
# Plotting countries with the largest number of unique customers.
fig = go.Figure(data=[go.Bar(name='Countries with Largest Number of Unique Customers',
                             x=retail_countries_customers_desc['Country'].astype(str),
                             y=retail_countries_customers_desc['Customer Count'],
                             marker_opacity=1,
                             marker={'color': retail_countries_customers_desc['Customer Count'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Countries with Largest Number of Unique Customers', title_x=0.45,
                  xaxis_title="Countries", yaxis_title="Number of Unique Customers", plot_bgcolor='white')
fig.show()

# Plotting countries with the smallest number of unique customers.
fig = go.Figure(data=[go.Bar(name='Countries with Smallest Number of Unique Customers',
                             x=retail_countries_customers_asc['Country'].astype(str),
                             y=retail_countries_customers_asc['Customer Count'],
                             marker_opacity=1,
                             marker={'color': retail_countries_customers_desc['Customer Count'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='Countries with Smallest Number of Unique Customers', title_x=0.45,
                  xaxis_title="Countries", yaxis_title="Number of Unique Customers", plot_bgcolor='white')
fig.show()

## Product Analysis

### Quantity Analysis
#### We analyze the top 10 bestselling products and the 10 products with the most returns.

In [ ]:
# Group by product descriptions to calculate the total quantity sold for each product.
retail_products = retail.groupby(['Description'], as_index=False)['Quantity'].agg('sum')
# Get the products with the least quantities sold (most returns).
retail_products_asc = retail_products.sort_values('Quantity').head(10)
# Get the products with the highest quantities sold (bestselling).
retail_products_desc = retail_products.sort_values('Quantity', ascending=False).head(10)

#### Plotting Bestselling Products and Products with Most Returns


In [ ]:
# Plotting the bestselling products.
fig = go.Figure(data=[go.Bar(name='10 Bestselling Products', x=retail_products_desc['Description'].astype(str), y=retail_products_desc['Quantity'], marker_opacity=1, marker={'color': retail_products_desc['Quantity'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='10 Bestselling Products', title_x=0.45, xaxis_title="Products", yaxis_title="Total Quantity Sold", plot_bgcolor='white')
fig.show()

# Plotting the products with the most returns.
fig = go.Figure(data=[go.Bar(name='10 Products with Most Returns', x=retail_products_asc['Description'].astype(str), y=retail_products_asc['Quantity'], marker_opacity=1, marker={'color': retail_products_asc['Quantity'], 'colorscale': 'Rainbow'})])
fig.update_layout(title='10 Products with Most Returns', title_x=0.45, xaxis_title="Products", yaxis_title="Total Quantity Returned", plot_bgcolor='white')
fig.show()

## Cohort Analysis

### Customer Retention Analysis
#### We perform a cohort analysis to understand customer retention over different periods.


In [ ]:
# Concert InvoiceDate to datetime
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'], format='%m/%d/%y %H:%M', errors='coerce')
# Extract quarter and year from InvoiceDate for cohort analysis.
retail['InvoiceQuarter'] = ('Q' + retail['InvoiceDate'].dt.quarter.astype(str) + '/' + retail['InvoiceDate'].dt.year.astype(str))
# Create a mapping of quarters for the cohort analysis.
quarters_map = dict(zip(retail['InvoiceQuarter'].unique(), range(len(retail['InvoiceQuarter'].unique()))))
# Map the invoice quarter to a unique ID for analysis.
retail['InvoiceQuarterID'] = retail['InvoiceQuarter'].map(quarters_map)
# Create cohort identifiers for each customer.
retail['CohortQuarterID'] = retail.groupby('Customer ID')['InvoiceQuarterID'].transform('min')
retail['CohortQuarter'] = retail['CohortQuarterID'].map(dict(zip(quarters_map.values(), quarters_map.keys())))
retail['CohortIndex'] = retail['InvoiceQuarterID'] - retail['CohortQuarterID']
# Aggregate customer count for each cohort.
cohort_retention = retail.groupby(['CohortQuarterID', 'CohortIndex'])['Customer ID'].apply(pd.Series.nunique).reset_index()
cohort_retention.rename(columns={'Customer ID': 'Customer Count'}, inplace=True)
cohort_retention_count = cohort_retention.pivot_table(index='CohortQuarterID', columns='CohortIndex', values='Customer Count')
# Map cohort quarters to proper names.
cohort_retention_count['Cohort Quarter'] = cohort_retention_count.index.map(dict(zip(quarters_map.values(), quarters_map.keys())))
cohort_retention_count = cohort_retention_count.set_index('Cohort Quarter')
cohort_size = cohort_retention_count.iloc[:, 0]
retention = cohort_retention_count.divide(cohort_size, axis=0)
retention = (retention * 100).round(2)
retention = retention.iloc[::-1]
# Plotting cohort retention rates.
fig = go.Figure(data=go.Heatmap(z=retention, y=retention.index, colorscale='Greens', text=retention, texttemplate="%{text}%", colorbar_title='Retention Rate, %', xgap=3, ygap=3))
fig.update_layout(title="Cohort Analysis: Retention Rate", xaxis_title="Cohorts", yaxis_title="Quarters", plot_bgcolor='white')
fig.show()

## Online Sales on Timeline
### Quarterly and Monthly Sales Analysis
#### We plot the total sales by quarter and by month to observe the trends.

In [ ]:
# Group by quarters to calculate total sales for each quarter.
retail_quarters_by_sales = retail.groupby(['InvoiceQuarterID'], as_index=False)['Total'].agg('sum')
# Map quarters to proper names for analysis.
retail_quarters_by_sales['InvoiceQuarter'] = retail_quarters_by_sales['InvoiceQuarterID'].map(dict(zip(quarters_map.values(), quarters_map.keys())))
# Plotting total sales by quarter.
fig = go.Figure(data=[go.Bar(name='Quarterly Sales', x=retail_quarters_by_sales['InvoiceQuarter'], y=retail_quarters_by_sales['Total'], marker_opacity=1, text='Total', marker={'color': retail_quarters_by_sales['Total'], 'colorscale': 'Portland'})])
fig.update_layout(title="Quarterly Sales", xaxis_title="Quarters", yaxis_title="Total Amount, £", plot_bgcolor='white')
fig.show()

In [ ]:
# Extract and convert invoice dates to monthly periods for analysis.
retail_monthly = retail.copy()
retail_monthly['InvoiceMonth'] = retail_monthly['InvoiceDate'].dt.to_period('M')
# Group by months to calculate total sales for each month.
retail_monthly_by_sales = retail_monthly.groupby(['InvoiceMonth'], as_index=False)['Total'].agg('sum')
# Plotting total sales by month.
fig = go.Figure(data=[go.Bar(name='Monthly Sales', x=retail_monthly_by_sales['InvoiceMonth'].astype(str), y=retail_monthly_by_sales['Total'], marker_opacity=1, marker={'color': retail_monthly_by_sales['Total'], 'colorscale': 'Portland'})])
fig.update_layout(title="Monthly Sales", xaxis_title="Months", yaxis_title="Total Amount, £", plot_bgcolor='white')
fig.show()


# Conclusion
### This analysis provides a comprehensive overview of the sales, customers, products, and retention rates of the online retail store. It helps to identify the most valuable customers, the bestselling products, seasonal trends, and countries contributing the most to sales.